In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import sys
import warnings
from pathlib import Path

module_path = Path('../..')
if module_path not in sys.path:
    sys.path.append(str(module_path.resolve()))

import mf2
import multiLevelCoSurrogates as mlcs

np.random.seed(20160501)  # Setting seed for reproducibility
OD = mf2.forrester

np.set_printoptions(linewidth=200, edgeitems=10, precision=4, suppress=True)
plot_dir = Path('../../plots/')
data_dir = Path('../../files/')

# EGO - 1D function

First creating an inverted function as BO is currently hardcoded for maximization problems

In [ ]:
inv_OD = mf2.MultiFidelityFunction(
    name='Inverted Forrester',
    u_bound=np.array(OD.u_bound), l_bound=np.array(OD.l_bound),
    functions=[lambda x: -OD.high(x), lambda x: -OD.low(x)],
    fidelity_names=['high', 'low'],
)

In [ ]:
plot_x = np.linspace(start=0,stop=1,num=501).reshape(-1,1)
low_x = np.linspace(0,1,6).reshape((-1,1))
high_x = low_x[[2,3]].reshape((-1,1))

archive = mlcs.CandidateArchive(ndim=1, fidelities=['high', 'low', 'high-low'])
archive.addcandidates(low_x, inv_OD.low(low_x), fidelity='low')
archive.addcandidates(high_x, inv_OD.high(high_x), fidelity='high')

np.random.seed(20160501)
mfbo = mlcs.MultiFidelityBO(inv_OD, archive, schema=[1,1])

# Plotting
fig, axes = plt.subplots(3,3, figsize=(12,9))

for idx, ax in enumerate(axes.flatten()):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        plot_hier, std_hier = mfbo.models['high'].predict(plot_x, mode='both')
    
    line_1, = ax.plot(plot_x, inv_OD.high(plot_x), label='high')
    line_2, = ax.plot(plot_x, inv_OD.low(plot_x), label='low')
    line_high, = ax.plot(plot_x, mfbo.direct_models['high'].predict(plot_x), label='high-fit GP')
    line_hier, = ax.plot(plot_x, mfbo.models['high'].predict(plot_x), label='co-kriging')
    scat_2 = ax.scatter(*archive.getcandidates(fidelity='low'), color=line_2.get_color())
    scat_1 = ax.scatter(*archive.getcandidates(fidelity='high'), color=line_1.get_color())
    
    ax.fill_between(plot_x.flatten(), plot_hier - 3*std_hier, plot_hier + 3*std_hier, alpha=.25, color=line_hier.get_color())
    
    ax2 = ax.twinx()
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        line_acq, = ax2.plot(plot_x, mfbo.utility(plot_x, gp=mfbo.models['high'], y_max=archive.max['high']),
                             alpha=.5, label='acq', color='C4')
        line_std_low, = ax2.plot(plot_x, mfbo.models['low'].predict(plot_x, mode='std'),
                                 alpha=.5, label='std low', color='C5', ls='--', )
        line_std_diff, = ax2.plot(plot_x, mfbo.models['high'].diff_model.predict(plot_x, mode='std'),
                                  alpha=.5, label='std diff', color='C6', ls=':', )
#     line_std, = ax2.plot(plot_x, std_hier, color='C7', label='std hierarchical')
    ax2.set_ylim(bottom=0)    
    
    lines = [
        line_1, line_2, line_high, line_hier, 
        line_std_low, line_std_diff, line_acq,
#         line_std,
    ]
    
    ax.set_title(f'Iteration {idx}')
    ax.set_xlim([0, 1])
    ax.set_ylim([-16, 10])
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax2.tick_params('y', colors='#555555')
    
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        mfbo.iteration(idx)

plt.tight_layout()
plt.subplots_adjust(bottom=0.1)
ax = fig.add_axes([0,0, 1,.05])
ax.axis('off')
ax.legend(
    lines, [l.get_label() for l in lines], ncol=4,
    loc='upper center', bbox_to_anchor=(.5,1),
)
    
plt.savefig(f'{plot_dir}1D_BO.png')
plt.savefig(f'{plot_dir}1D_BO.pdf')
plt.show()